In [8]:
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import importlib
import scripts.variables as var


"""
To beat:

ACCURACY:  0.882525697503671
RECALL:  0.1708008504606662
F1:  0.25078043704474506

"""



importlib.reload(var)


print('DONE')

DONE


In [10]:
bank=pd.read_csv(var.pathname)
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,NaN,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,0.0,1.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,0.0,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,0.0,0.0,1.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [15]:
columns_with_nulls=list(bank.isnull().sum()[bank.isnull().sum()>0].index)
columns_with_nulls

['default', 'housing', 'loan']

In [19]:
#Default
bank['default'].value_counts(dropna=False)
#most likely outcome is that this columns does not matter. No information about default.
#it is likely that they have no credit open. Will asign -1
bank['default'].fillna(-1,inplace=True)

In [20]:
#Housing
bank['housing'].value_counts(dropna=False)

housing
1.0    21576
0.0    18622
NaN      990
Name: count, dtype: int64

In [22]:
#Loan
bank['loan'].value_counts(dropna=False)

loan
0.0    33950
1.0     6248
NaN      990
Name: count, dtype: int64

In [28]:
##Dropping nulls for housing and loan. Consistent rows. Two birds one shot
bank[(bank.loan.isnull())&(bank.housing.isnull())].shape[0]
bank=bank.dropna(subset=['housing', 'loan'])
bank.isnull().sum().sum()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,-1.0,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,0.0,1.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,0.0,0.0,0.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,0.0,0.0,1.0,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,0.0,1.0,0.0,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,blue-collar,married,professional.course,0.0,0.0,0.0,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,retired,married,university.degree,0.0,1.0,0.0,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,technician,married,professional.course,0.0,0.0,0.0,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


In [100]:
##CategoricalColumns
dtypes_df=pd.DataFrame(bank.dtypes, columns=['dataType'])

categorical_columns=list(dtypes_df[dtypes_df.dataType=='object'].index)
pending_columns=[col for col in bank.columns if col not in categorical_columns]
numerical_columns=[]
for col in pending_columns:
    #print(col.upper())
    if bank[col].nunique() <5:
        #print(col.upper(), bank[col].unique())
        categorical_columns.append(col)
        #print("CAT",categorical_columns)
    else:
        numerical_columns.append(col)
        #print("NUM",numerical_columns)
categorical_columns, numerical_columns

(['job',
  'marital',
  'education',
  'contact',
  'month',
  'day_of_week',
  'poutcome',
  'default',
  'housing',
  'loan',
  'y'],
 ['age',
  'campaign',
  'pdays',
  'previous',
  'emp.var.rate',
  'cons.price.idx',
  'cons.conf.idx',
  'euribor3m',
  'nr.employed'])

In [101]:
for col in numerical_columns:
    
    print(bank[col].nunique(),col.upper())
    

78 AGE
41 CAMPAIGN
27 PDAYS
8 PREVIOUS
10 EMP.VAR.RATE
26 CONS.PRICE.IDX
26 CONS.CONF.IDX
316 EURIBOR3M
11 NR.EMPLOYED


In [118]:
bank['nr.employed'].value_counts()

nr.employed
5228.1    15850
5099.1     8348
5191.0     7543
5195.8     3616
5076.2     1613
5017.5     1046
4991.6      755
5008.7      636
4963.6      615
5023.5      166
5176.3       10
Name: count, dtype: int64

In [109]:
numerical_columns

['age',
 'campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']